#### Manual testing...

This notebook serves as an integration test to confirm that the model is behaving as we expect.

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib inline

In [2]:
MODEL_DIR = os.path.realpath(os.path.dirname('..\superscript_model'))
sys.path.append(os.path.normpath(MODEL_DIR))

In [3]:
from superscript_model import model
from superscript_model.utilities import Random

In [4]:
abm = model.SuperScriptModel(1000)

In [ ]:
steps = range(1000)
active_projects = []
team_size = []

for step in steps:
    
    start_time = time.time()
    abm.step()
    elapsed_time = time.time() - start_time  
    print("Step %d took %.2f seconds" % (step, elapsed_time))
    
    size = 0
    active_projects.append(abm.inventory.active_count)
    
    for project in abm.inventory.projects.values():
        size += len(project.team.members)
    team_size.append(size/len(abm.inventory.projects.values()))

Step 0 took 1.07 seconds
Step 1 took 1.53 seconds
Step 2 took 1.53 seconds
Step 3 took 1.71 seconds
Step 4 took 1.85 seconds
Step 5 took 1.82 seconds
Step 6 took 1.94 seconds
Step 7 took 1.82 seconds
Step 8 took 1.78 seconds
Step 9 took 2.16 seconds
Step 10 took 2.15 seconds
Step 11 took 2.45 seconds
Step 12 took 2.00 seconds
Step 13 took 2.12 seconds
Step 14 took 2.13 seconds
Step 15 took 2.37 seconds
Step 16 took 2.51 seconds
Step 17 took 2.05 seconds
Step 18 took 2.06 seconds
Step 19 took 1.93 seconds
Step 20 took 1.96 seconds
Step 21 took 2.13 seconds
Step 22 took 2.16 seconds
Step 23 took 2.73 seconds
Step 24 took 2.01 seconds
Step 25 took 2.07 seconds
Step 26 took 2.15 seconds
Step 27 took 2.16 seconds
Step 28 took 2.21 seconds
Step 29 took 2.02 seconds
Step 30 took 2.27 seconds
Step 31 took 2.36 seconds
Step 32 took 2.19 seconds
Step 33 took 2.03 seconds
Step 34 took 2.05 seconds
Step 35 took 1.90 seconds
Step 36 took 2.05 seconds
Step 37 took 1.91 seconds
Step 38 took 1.80 seco

Step 308 took 2.20 seconds
Step 309 took 2.07 seconds
Step 310 took 1.99 seconds
Step 311 took 2.16 seconds
Step 312 took 1.69 seconds
Step 313 took 2.23 seconds
Step 314 took 2.23 seconds
Step 315 took 1.85 seconds
Step 316 took 2.35 seconds
Step 317 took 1.57 seconds
Step 318 took 2.27 seconds
Step 319 took 2.38 seconds
Step 320 took 1.65 seconds
Step 321 took 0.98 seconds
Step 322 took 1.46 seconds
Step 323 took 1.99 seconds
Step 324 took 1.03 seconds
Step 325 took 1.21 seconds
Step 326 took 2.08 seconds
Step 327 took 2.14 seconds
Step 328 took 2.09 seconds
Step 329 took 2.17 seconds
Step 330 took 2.02 seconds
Step 331 took 2.19 seconds
Step 332 took 1.81 seconds
Step 333 took 2.14 seconds
Step 334 took 2.11 seconds
Step 335 took 2.20 seconds
Step 336 took 1.87 seconds
Step 337 took 2.28 seconds
Step 338 took 2.25 seconds
Step 339 took 1.94 seconds
Step 340 took 2.18 seconds
Step 341 took 2.20 seconds
Step 342 took 2.12 seconds
Step 343 took 2.20 seconds
Step 344 took 2.23 seconds
S

Step 612 took 2.08 seconds
Step 613 took 2.11 seconds
Step 614 took 2.22 seconds
Step 615 took 2.29 seconds
Step 616 took 1.69 seconds
Step 617 took 2.15 seconds
Step 618 took 1.91 seconds
Step 619 took 1.95 seconds
Step 620 took 1.72 seconds
Step 621 took 2.40 seconds
Step 622 took 2.26 seconds
Step 623 took 2.47 seconds
Step 624 took 2.18 seconds
Step 625 took 2.15 seconds
Step 626 took 1.86 seconds
Step 627 took 1.32 seconds
Step 628 took 2.07 seconds
Step 629 took 2.04 seconds
Step 630 took 2.15 seconds
Step 631 took 1.96 seconds
Step 632 took 2.16 seconds
Step 633 took 2.20 seconds
Step 634 took 2.31 seconds
Step 635 took 2.35 seconds
Step 636 took 2.14 seconds
Step 637 took 2.07 seconds
Step 638 took 2.23 seconds
Step 639 took 2.05 seconds
Step 640 took 2.07 seconds
Step 641 took 1.93 seconds
Step 642 took 1.75 seconds
Step 643 took 2.05 seconds
Step 644 took 1.99 seconds
Step 645 took 1.95 seconds
Step 646 took 2.31 seconds
Step 647 took 2.22 seconds
Step 648 took 2.06 seconds
S

In [ ]:
plt.subplot(2,1,1)
plt.plot(steps, active_projects)
plt.ylabel('active projects')

plt.subplot(2,1,2)
plt.plot(steps, team_size)
plt.xlabel('time steps')
plt.ylabel('average team size');

In [ ]:
print(np.mean(active_projects[200:]))
print(np.mean(team_size[200:]))

#### We now select a random worker from the model and inspect the worker:

This produces a json formatted text summary of the worker. Note that this representation will be updated when we add in the worker's history and project activities.

In [ ]:
random_worker = Random.choice(abm.schedule.agents)

print(random_worker.skills.to_string())

#### We now do the same for a randomly selected project..

In [ ]:
random_project = Random.choice(list(abm.inventory.projects.values()))

In [ ]:
print(random_project.requirements.to_string())

#### For this project we can also inspect the Team:

### Note: here the team of 5 is doing its best to meet the high skill unit requirements...Check with Michael. 

In [ ]:
print(random_project.team.to_string())

#### Note that allocating the total skill units across the skills in a robust way was a non-trivial problem! Solved as follows...

*(Move this to another notebook).*

Select 5 numbers in (1,10) that sum to X, where X <=50
Always a = max(a,1) and b = min(b,10)
-	Select n1 randint(a,b) where a = X – (4\*10) and b = X – (4\*1)
-	Select n2 randint(a,b) where a = X – n1 – (3\*10) and b = X – n1 – (3\*1)
-	Select n3 randint(a,b) where a = X – n1 – n2 – (2\*10) and b = X – n1 – n2 – (2\*1)
-	…
-	Select n5 randint(a,b) where a = X – n1 – n2 – n3 – n4 and b = X – n1 – n2 – n3 – n4


In [ ]:
def generate_skills(total):

    min_units = 1
    max_units = 10

    remaining = total
    length = 5

    assert total <= length*max_units
    assert total >= length*min_units

    skills = []

    for i in range(length):

        a = remaining - (length - (i+1))*max_units
        a = max(a, min_units)

        b = remaining - (length - (i+1))*min_units
        b = min(b, max_units)

        n = Random.randint(a,b)
        skills.append(n)
        remaining -= n
        
    assert sum(skills) == total
    return skills

In [ ]:
for t in np.arange(5,50):
    print(generate_skills(t), "; sum = ", t)

In [ ]:
from itertools import combinations

In [ ]:
list(combinations([1], 2))

In [ ]:
test = {1:2}

In [ ]:
Random.choices([1], 3)

In [ ]:
test = np.int32(1)
t = {test: 1}

In [ ]:
test = {1:2, 3:4}

In [ ]:
max(0,3)

In [ ]:
len(range(10))

In [ ]:
random_worker.model.schedule.steps